In [55]:
import pandas as pd

#Leemos el archivo inicial.
pathLectura = "C:\\Python\\Fifo\\fifo.xlsx"
# Si querés ingresar un archivo por comando, simplemente descomentá la siguiente línea
# pathEscritura = input("Ingresá el camino para el archivo a leer")

#Acomodá el df o el archivo según tu necesidad

data = pd.read_excel(pathLectura,"Sheet1")

df = pd.DataFrame(data)
df['Ganancia'] = 0

#Reemplazamos para operar más fácil
df['Descripción'] = df['Descripción'].replace(r'.*Rescate.*', 'Rescate', regex=True)
df['Descripción'] = df['Descripción'].replace(r'.*Suscripción.*', 'Suscripción', regex=True)

#Orden por fecha ascendente
df = df.sort_values(by='Fecha',ascending=True)

#Reset de index
df.reset_index(inplace=True, drop=True)

df

,Fecha,Nro. de Operación,Descripción,Valor Cuota Parte,Importe,Cuotapartes,Ganancia
0,2021-03-09,107055,Suscripción,5.576474,1000.00,179.3247,0
1,2021-12-30,201236,Suscripción,7.133638,1100000.00,154199.0215,0
2,2022-01-03,282060,Rescate,7.156648,1104831.48,-154378.3462,0
3,2022-01-13,208378,Suscripción,7.215105,2800000.00,388074.7404,0
4,2022-01-14,289144,Rescate,7.221030,2802299.34,-388074.7404,0
...,...,...,...,...,...,...,...
121,2022-12-23,604195,Rescate,10.945095,1500000.00,-137047.6912,0
122,2022-12-27,429753,Suscripción,11.018211,3000000.00,272276.5065,0
123,2022-12-28,609092,Rescate,11.036586,2200000.00,-199337.0051,0
124,2022-12-28,609093,Rescate,11.036586,500000.00,-45303.8648,0


In [56]:
# Crear columna para la cantidad de cuotapartes restantes por suscripción
df['Cantidad restante'] = 0

# Iterar a través de cada fila del DataFrame
for index, row in df.iterrows():

    # Si la fila es una suscripción, se copia el valor a cuotapartes restantes
    if row['Descripción'] == 'Suscripción':
        #Vamos trackeando lo que queda
        df.loc[index,'Cantidad restante'] = abs(row['Cuotapartes'])
    # Si la fila es un rescate, vender cuotapartes
    elif row['Descripción'] == 'Rescate':
        cuotapartes_vendidas = abs(row['Cuotapartes'])

        # Hago un dataframe sólo con subscripciones restantes.
        df2 = df[df['Cantidad restante']>0]

        # Vender cuotapartes en orden FIFO
        while cuotapartes_vendidas > 0:
            # Arrancamos con este valor sin considerar las subscripciones
            ganancia = cuotapartes_vendidas * row['Valor Cuota Parte']

            # Iteramos sólo por aquellos que tengan cantidad restante. Indexrescate en este caso, es el mismo índice que el dataframe original.
            for indexrescate, rowrescate in df2.iterrows():
                if (cuotapartes_vendidas > 0):
                    # Si es igual o mayor, la ganancia simplemente será la resta entre uno y otro
                    if(float (int (rowrescate['Cantidad restante'] * 10000) / 10000) >= float (int (cuotapartes_vendidas * 10000) / 10000)):
                        ganancia -= (cuotapartes_vendidas * rowrescate['Valor Cuota Parte'])
                        df.loc[indexrescate, 'Cantidad restante'] -= cuotapartes_vendidas
                        #Limpieza de números muy chicos
                        if(abs(df.loc[indexrescate, 'Cantidad restante']) < 0.001):
                            df.loc[indexrescate, 'Cantidad restante'] = 0
                        # Set en cero para salir
                        cuotapartes_vendidas = 0
                    # Si es menor, el cálculo cambia un poco, ya que se vende el remanente del rescate
                    elif(float (int (rowrescate['Cantidad restante'] * 10000) / 10000) < float (int (cuotapartes_vendidas * 10000) / 10000)):
                        ganancia -= (rowrescate['Cantidad restante'] * rowrescate['Valor Cuota Parte'])
                        df.loc[indexrescate, 'Cantidad restante'] = 0
                        cuotapartes_vendidas -= rowrescate['Cantidad restante'] 

        #Escribimos la celda
        df.loc[index, 'Ganancia'] = ganancia

#Escribimos el archivo final.
pathEscritura = "C:\\Python\\Fifo\\fifo2.xlsx"
# Si querés ingresar un archivo por comando, simplemente descomentá la siguiente línea
# pathEscritura = input("Ingresá el camino para el nuevo archivo")
df.to_excel(pathEscritura,"Sheet1")